# 정확도(Accuracy)

정확도는 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표입니다.

$ 정확도(Accuracy) = (예측 결과가 동일한 데이터 건수) / (전체 예측 데이터 건수) $

직관적인 성능을 나타내는 지표이나, 이진 분류의 경우 데이터에 따라 성능을 왜곡할 수도 있습니다. 따라서 정확도만 가지고 성능을 평가하지 않습니다.

어떻게 왜곡하는지 예를 통해 살펴보겠습니다.

타이타닉 예제에서 ML 알고리즘 적용 시 예측 정확도가 80%대 입니다. 탑승객이 여자일 때 생존 확률이 높기 때문에 별다른 알고리즘 적용 없이 무조건 성별이 여자인 경우 생존, 남자인 경우 사망으로 예측 결과를 예측해도 비슷한 수치가 나올 수 있습니다. 

In [1]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass
    # predicet() 메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함.
    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred

위 코드는 사이킷런의 BaseEstimator 클래스를 상속받아 아무 학습을 하지 않고, 성별에 따라 생존자를 예측하는 단순한 분류기를 생성합니다. BaseEstimator를 상속받으면 개발작아 커스터마이징을 할 수 있습니다.

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('./train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는: 0.7877


이렇게 단순한 알고리즘으로 예측을 하더라도 데이터의 구성에 따라 정확도 결과는 약 78.77%로 꽤 높은 수치가 나올 수 있기에 정확도를 평가 지표로 사용할 때는 매우 신중해야 합니다.

이번에는 MNIST 데이터셋을 변환헤 불균형한 데이터 셋으로 만든 후 어떤 문제가 발생할 수 있는지 살펴보겠습니다.

MNIST 데이터셋은 0 부터 9의 숫자 이미지의 픽셀 정보를 가지고 있습니다.  레이블 값이 0부터 9까지 있는 멀티 레이블 분류 셋을 7인 것만 True, 나머지는 False로 변환해 이진 분류 문제로 변형해보겠습니다.

즉 전체 데이터의 10%만 True입니다.

이런 불균형한 데이터셋에 모든 데이터를 False로 예측하는 분류기를 이용해 정확도를 측정하면 90%에 가까운 예측 정확도를 나타냅니다.

데이터 분포도가 불균일하면 높은 수치가 나타날 수 있는 것이 정확도 평가 지표의 맹점입니다.

In [12]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X),1), dtype=bool)

# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True 이고, 이를 astype(int)로 1로 변환, 7번이 아니면 False 0 으로 변환
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [16]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기:', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)

fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


단순히 predict()의 결과를 np.zeros()를 이용해 모두 0으로 반환해도 예측 정확도는 90%입니다. 

이처럼 정확도 평가 지표는 불균형한 레이블 데이터셋에서는 성능 수치로 사용되어서는 안됩니다.